# Fine-Tuning Neural Network Hyperparameters

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [2]:
# Dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

scaler = StandardScaler()
X_train_full = scaler.fit_transform(X_train_full)
X_test = scaler.transform(X_test)

In [3]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=0.001, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [4]:
# transfer the keras model to a scikit-learn regressor
# so we can utilise the helper functions provided by scikit-learn
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [5]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100).tolist(), 
    "learning_rate": [1e-4, 1e-3, 1e-2],
}

In [9]:
rnd_search_cv = RandomizedSearchCV(
                        estimator=keras_reg, 
                        param_distributions=param_distributions, 
                        n_iter=1, 
                        cv=3)
rnd_search_cv.fit(X_train_full, y_train_full, epochs=1)

484/484 [==============================] - 0s 375us/step - loss: 0.8214


RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fd9b08e56d0>,
                   n_iter=1,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01],
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...]})

In [10]:
rnd_search_cv.best_params_

{'n_neurons': 19, 'n_hidden': 1, 'learning_rate': 0.01}

In [11]:
rnd_search_cv.best_score_

-0.5653369426727295

In [12]:
model = rnd_search_cv.best_estimator_.model